# 提高纯平行数据集世界坐标系统的准确性

## 引言

在本笔记本中，我们将介绍提高JWST在安装DMS 10.2（2024年4月）之前观察的纯平行数据集头部世界坐标系统（WCS）准确性所需的步骤。示例数据集为`jw01571078001_03201_00001_nis_rate.fits`，这是来自纯平行项目ID 1571（首席研究员：Malkan）的直接图像。请注意，必须在纯平行访问期间拍摄的所有单个数据集上运行`update_parallel_wcs.py`脚本，以获取该访问的改进WCS。

在本笔记本中，我们假设所有相关文件都位于当前工作目录中。

### 安装管道和其他所需包

所需的包在提供的文件`requirements.txt`中。我们通常建议创建一个新的conda环境，然后安装这些所需的包：

conda create -n improve_pure_parallel_wcs

conda activate improve_pure_parallel_wcs

pip install -r requirements.txt

最后发布日期：2025年2月4日

### 导入

下一单元中的导入是运行本笔记本所需的。

In [ ]:
import copy  # 导入copy模块，用于对象的浅拷贝和深拷贝

import jwst.datamodels as dm  # 导入JWST数据模型模块

import numpy as np  # 导入NumPy库，用于数值计算

from astropy.io import ascii  # 从Astropy库导入ascii模块，用于处理ASCII文件

from astropy import units as u  # 从Astropy库导入units模块，用于单位转换

from astropy.coordinates import SkyCoord  # 从Astropy库导入SkyCoord类，用于天球坐标处理

from astropy.table import Table  # 从Astropy库导入Table类，用于处理表格数据

from astropy.time import Time  # 从Astropy库导入Time类，用于时间处理

from astropy.convolution import interpolate_replace_nans, Gaussian2DKernel  # 导入插值和高斯核函数

import astropy.io.fits as pyfits  # 导入FITS文件处理模块

import matplotlib.pyplot as plt  # 导入Matplotlib库的pyplot模块，用于绘图

import matplotlib.patches as patches  # 导入Matplotlib库的patches模块，用于绘制形状

import pysiaf  # 导入pysiaf库，用于处理SIAF（科学仪器适配框架）

import warnings  # 导入warnings模块，用于处理警告信息

import urllib.request  # 导入urllib.request模块，用于处理URL请求

from astroquery.gaia import Gaia  # 从astroquery库导入Gaia模块，用于查询Gaia数据

%matplotlib inline  # 在Jupyter Notebook中内联显示Matplotlib图形

### 定义几个函数

In [ ]:
# 这个示例展示了一个纯平行成像数据集的世界坐标系统（WCS）的（误）对齐情况
# 该数据集是在安装DMS 10.2之前获取的，相对于GAIA DR3，并且展示了
# 应用“update_parallel_wcs.py”脚本如何改善该对齐。

# 首先一些函数用于获取某个天空位置附近的GAIA星表：

def query_gaia(ra, dec, radius, verbose=False, epoch=None, filename=None):
    """
    使用astroquery执行GAIA DR3查询，返回源表。

    参数
    ----------
    ra:   浮点值，右升角（RA），单位为十进制度
    dec:  浮点值， declination（DEC），单位为十进制度
    radius:  可选浮点值，搜索半径，单位为弧秒，默认值为1.0
    epoch:   可选浮点值，位置的历元，单位为十进制年份
             （例如2023.197588611，表示2021-05-24T17:46:12.814）
             如果给定，则返回的位置会考虑自行运动
    verbose:  可选布尔值，如果为True，则打印源列表
    filename:  可选字符串值，作为输出文件名，如果verbose标志被设置（如果为None，则仅打印到终端）

    返回
    -------
    gaiadata1:  一个Astropy表，包含源的目录
    """
    coord = SkyCoord(ra=ra, dec=dec, unit=(u.degree, u.degree), frame='icrs')  # 创建天空坐标对象
    radius = u.Quantity(radius/3600.0, u.deg)  # 将半径转换为度
    Gaia.ROW_LIMIT = -1  # 设置GAIA查询的行限制
    Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"  # 设置主要GAIA表
    gaiadata1 = Gaia.query_object_async(coordinate=coord, width=radius*2, height=radius*2)  # 异步查询GAIA数据

    if verbose:  # 如果verbose为True
        gphot = gaiadata1['phot_g_mean_mag']  # 获取G波段的平均光度
        gbphot = gaiadata1['phot_bp_mean_mag']  # 获取BP波段的平均光度
        grphot = gaiadata1['phot_rp_mean_mag']  # 获取RP波段的平均光度
        parallax = gaiadata1['parallax']  # 获取视差
        names = gaiadata1['designation']  # 获取星体名称

        with warnings.catch_warnings():  # 捕获警告
            warnings.simplefilter("ignore")  # 忽略警告
            gabs = gphot - 5.*np.log10(1000./parallax) + 5.  # 计算绝对光度
            gcol = gbphot - grphot  # 计算颜色指数

        if epoch is None:  # 如果没有给定历元
            epoch = 2016.0  # 默认历元为2016年

        gaiadata2 = apply_precession(gaiadata1, epoch-2016.0)  # 应用岁差修正

        ra = gaiadata2['ra']  # 获取修正后的RA
        dec = gaiadata2['dec']  # 获取修正后的DEC

        with warnings.catch_warnings():  # 捕获警告
            warnings.simplefilter("ignore")  # 忽略警告
            for ind in range(len(ra)):  # 遍历每个源
                g_col = gcol[ind] if (gbphot[ind] < 90.) and (grphot[ind] < 90.) else 0.0  # 计算颜色指数
                line = f"{ra[ind]:12.8f} {dec[ind]:13.8f} {gphot[ind]:10.6f} {gbphot[ind]:10.6f} {grphot[ind]:10.6f} {parallax[ind]:10.3f} {gabs[ind]:10.4f} {g_col:10.4f} '{names[ind]}'"  # 格式化输出行
                line = line.replace('nan', '0.0')  # 替换nan为0.0

            if filename:  # 如果提供了文件名
                with open(filename, 'w') as outfile:  # 打开文件以写入
                    print('          RA           DEC          g        gbp        grp   parallax      abs_g    gbp_grp               GAIA_Designation', file=outfile)  # 写入表头
                    for ind in range(len(ra)):  # 遍历每个源
                        g_col = gcol[ind] if (gbphot[ind] < 90.) and (grphot[ind] < 90.) else 0.0  # 计算颜色指数
                        line = f"{ra[ind]:12.8f} {dec[ind]:13.8f} {gphot[ind]:10.6f} {gbphot[ind]:10.6f} {grphot[ind]:10.6f} {parallax[ind]:10.3f} {gabs[ind]:10.4f} {g_col:10.4f} '{names[ind]}'"  # 格式化输出行
                        line = line.replace('nan', '0.0')  # 替换nan为0.0
                        print(line, file=outfile)  # 写入数据行

    return gaiadata2  # 返回修正后的GAIA数据

def apply_precession(catalog, deltat):
    """
    应用岁差修正以更新值的目录。使用astropy.SkyCoord
    apply_space_motion更新位置。

    参数：
    catalog:     一个numpy表类型变量，包含元素'ra'，'dec'，'pmra'，
                 和'pmdec'，根据GAIA DR3目录；天空坐标
                 必须以度为单位， proper motions必须以毫角秒/年为单位
    deltat:      浮点值，运动的时间变化，单位为十进制年份

    返回
    -------
    newcatalog:  目录的副本，包含'newra'，'newdec'元素，修正后的天空位置；
                 如果没有可用的自行运动数据，则newra = ra和newdec = dec
    """
    newcatalog = copy.deepcopy(catalog)  # 深拷贝目录
    newcatalog['newra'] = catalog['ra']  # 初始化新RA
    newcatalog['newdec'] = catalog['dec']  # 初始化新DEC

    with warnings.catch_warnings():  # 捕获警告
        warnings.simplefilter("ignore")  # 忽略警告
        sky_coords = SkyCoord(catalog['ra'], catalog['dec'],  # 创建天空坐标对象
                              unit=(u.deg, u.deg),
                              pm_ra_cosdec=catalog['pmra'],  # 设置RA方向的自行运动
                              pm_dec=catalog['pmdec'],  # 设置DEC方向的自行运动
                              obstime=Time('2016-01-01 00:00:00.0'))  # 设置观测时间
        newpos = sky_coords.apply_space_motion(dt=deltat*u.yr)  # 应用空间运动

    newra = newpos.ra.value  # 获取新RA值
    newdec = newpos.dec.value  # 获取新DEC值
    newcatalog['newra'] = newra  # 更新新RA
    newcatalog['newdec'] = newdec  # 更新新DEC

    inds = np.isnan(newra)  # 查找新RA中的NaN值
    newcatalog['newra'][inds] = newcatalog['ra'][inds]  # 替换NaN为原RA
    newcatalog['newdec'][inds] = newcatalog['dec'][inds]  # 替换NaN为原DEC

    return newcatalog  # 返回修正后的目录

### 获取纯平行图像指向周围的GAIA目标的天空坐标

In [ ]:
# 现在获取NIRISS纯平行图像jw01571078001_03201_00001_nis_rate.fits周围的列表

# 从MAST档案下载。

# 注意：在这里下载或创建的文件将放置在当前工作目录中。

boxlink = 'https://stsci.box.com/shared/static/ydxn3hhndwup0qr85fuyqro6suufa6fx.fits'  # 文件链接

boxfile = 'jw01571078001_03201_00002_nis_rate.fits'  # 本地文件名

try:
    # 尝试下载文件
    urllib.request.urlretrieve(boxlink, boxfile)  # 从链接下载文件到本地

except Exception as e:
    # 如果下载失败，打印错误信息
    print(f"Error downloading file: {e}")

try:
    # 尝试打开FITS文件
    imfile = pyfits.open(boxfile)  # 使用pyfits打开下载的FITS文件

    hdr1 = imfile[1].header  # 获取文件头信息

    ra, dec = hdr1['CRVAL1'], hdr1['CRVAL2']  # 从头信息中提取RA和DEC坐标

    print(f'RA = {ra}, DEC = {dec}')  # 打印RA和DEC坐标

except Exception as e:
    # 如果打开文件失败，打印错误信息
    print(f"Error opening FITS file: {e}")

# 查询GAIA数据库，获取指定坐标附近的天体信息
mygaia = query_gaia(ra, dec, 65., verbose=True, epoch=2022.9968, filename='gaiacoords.out')  # 查询半径为65弧秒的GAIA数据

### 将这些天空坐标转换为图像中的 (x, y) 坐标，依据其 WCS，然后在图像上显示它们

In [ ]:
t = ascii.read('gaiacoords.out')  # 读取天体坐标数据

ra, dec = (t['RA'], t['DEC'])  # 提取RA和DEC列

ratefile = 'jw01571078001_03201_00002_nis_rate.fits'  # 指定JWST的速率文件

instr = imfile[0].header['INSTRUME']  # 获取仪器名称

aperture = imfile[0].header['APERNAME']  # 获取光圈名称

siaf = pysiaf.Siaf(instr)  # 创建SIAF对象

myaper = siaf[aperture]  # 获取指定光圈的SIAF信息

mod = dm.open(ratefile)  # 打开速率文件

# 计算姿态矩阵
am = pysiaf.utils.rotations.attitude_matrix(0, 0, mod.meta.pointing.ra_v1,
                                            mod.meta.pointing.dec_v1,
                                            mod.meta.pointing.pa_v3)

myaper.set_attitude_matrix(am)  # 设置光圈的姿态矩阵

x, y = myaper.sky_to_sci(ra, dec)  # 将天球坐标转换为科学坐标

# 创建包含RA, DEC, x, y的表格
xytab = Table([ra, dec, x, y], names=('RA', 'DEC', 'x', 'y'))

# 设置表格中各列的格式
xytab['RA'].info.format = '.8f'  # RA格式为8位小数
xytab['DEC'].info.format = '.8f'  # DEC格式为8位小数
xytab['x'].info.format = '.4f'  # x格式为4位小数
xytab['y'].info.format = '.4f'  # y格式为4位小数

print(xytab)  # 打印坐标表格

ys, xs = imfile[1].data.shape  # 获取图像数据的形状

fig = plt.figure(figsize=(15, 15))  # 创建图形窗口

ax = fig.add_subplot(111)  # 添加子图

xsize, ysize = (20, 20)  # 设置椭圆的大小

# 遍历每个坐标点
for i in range(len(t)):
    ax.add_patch(patches.Ellipse(  # 在图中添加椭圆
        (x[i], y[i]),  # 椭圆中心坐标
        (xsize),  # 椭圆宽度
        (ysize),  # 椭圆高度
        fill=False, color='red'))  # 椭圆不填充，颜色为红色

# 创建图像并对坏像素进行插值（用于显示）
kernel = Gaussian2DKernel(x_stddev=2)  # 创建高斯核
fixed_image = interpolate_replace_nans(imfile[1].data, kernel)  # 对坏像素进行插值

# 显示图像
ax.imshow(fixed_image, cmap='binary', origin='lower', extent=[0, xs-1, 0, ys-1], vmin=0.7, vmax=2)

请注意，WCS（世界坐标系统）偏差超过一个像素，这对WFSS（宽场光谱扫描）数据的光谱提取造成了问题，因为**该提取完全依赖于WCS的准确性**。

### 现在应用update_parallel_wcs.py脚本来修正WCS。请注意脚本计算出的像素坐标偏移：

In [ ]:
# 此单元假设您在工作目录中有脚本 update_parallel_wcs.py。

# 注意：在此笔记本外，应该在每个 _rate.fits 文件上运行此脚本，适用于纯平行访问：

# python update_parallel_wcs.py my_rate.fits <verbosity>

# 在对输入文件运行脚本后，可以（重新）运行 JWST 校准管道的第二阶段

# (`calwebb_image2` 和/或 `calwebb_spec2`)，这些文件现在将导致数据头中的 WCS 正确

# （并且在纯平行 WFSS 数据的情况下，光谱提取的位置也会被修正）。

#

# 默认情况下，当脚本运行时，`update_parallel_wcs.py` 会显示 CRVAL1/2 关键字的输入和输出值

# 如下所示。通过将可选参数 <verbosity> 设置为其他值，可以避免此行为。

#

# 该脚本通过在工作目录中创建一个名为 "pure_parallel_wcs_logfile" 的日志文件来跟踪其执行情况。

%run update_parallel_wcs.py jw01571078001_03201_00002_nis_rate.fits  # 运行更新平行 WCS 的脚本，处理指定的 FITS 文件

### 计算更新后的fits文件中GAIA目标的(x, y)坐标并打印更新后的(x, y)坐标：

In [ ]:
with pyfits.open('jw01571078001_03201_00002_nis_rate.fits') as imfile:  # 打开JWST的FITS文件

    hdr0 = imfile[0].header  # 获取文件的头信息

    myaper = pysiaf.Siaf(hdr0['INSTRUME'])[hdr0['APERNAME']]  # 根据仪器名称和光圈名称获取SIAF信息

    mod = dm.open('jw01571078001_03201_00002_nis_rate.fits')  # 打开数据模型

    am = pysiaf.utils.rotations.attitude_matrix(0, 0, mod.meta.pointing.ra_v1,  # 计算姿态矩阵
                                                mod.meta.pointing.dec_v1,
                                                mod.meta.pointing.pa_v3)

    myaper.set_attitude_matrix(am)  # 设置光圈的姿态矩阵

    newx, newy = myaper.sky_to_sci(ra, dec)  # 将天球坐标转换为科学坐标

    xytab = Table([x, y, newx, newy], names=('x', 'y', 'x_corr', 'y_corr'))  # 创建包含原始和校正坐标的表格

    for col in xytab.colnames:  # 遍历表格的列名

        xytab[col].info.format = '.4f'  # 设置每列的格式为小数点后四位

    print(xytab)  # 打印表格内容

# 实际上，用户会对给定目标的所有_rate.fits文件运行该脚本， 
# 然后对生成的图像运行第二阶段和第三阶段的管道 
# （具体来说，首先对直接图像运行Image2Pipeline，然后运行Image3Pipeline以创建组合 
# 图像和源目录，接着对光栅图像运行第二阶段和第三阶段的管道 
# （分别为Spec2Pipeline和Spec3Pipeline）。

In [ ]:
看起来您没有提供任何代码。如果您能提供需要注释的Python代码，我将很乐意为您添加中文注释。请将代码粘贴在这里。